In [1]:
import json
from os.path import join
import os
os.chdir("/home/tiagoalmeida/bioASQ-taskb/")

import sys
import pickle
import gc
import numpy as np
import time

##add keras to the modules
module_path = os.path.abspath(os.path.join('pubmed_data'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from pubmed_data import pubmed_helper as ph

from models.generic_model import ModelAPI, f_recall, f_map

import


In [2]:
bioASQ_data_path = "/backup/BioASQ-training7b/"
bioASQ_data_train = json.load(open(os.path.join(bioASQ_data_path,"7b_train_split.json")))
bioASQ_data_test = json.load(open(os.path.join(bioASQ_data_path,"7b_test_split.json")))

#goldstandard
bioASQ_test_expectation = list(map(lambda x:x["documents"],bioASQ_data_test))

In [48]:
from models.generic_model import AWE,AWE_TFIDF

#MODE = "regex_full_tokens"
MODE = "regex_less_700k_freq"

#Load
tk = ph.load_tokenizer(MODE)
model = AWE.load(MODE+"_title_abs")

model.num_docs = 18824355

Load regex_less_700k_freq_tokenizer.p


In [4]:
#Test data
bioASQ_test_queries_tokenized = tk.texts_to_sequences(map(lambda x:x["body"],bioASQ_data_test))
bioASQ_test_expectation = list(map(lambda x:x["documents"],bioASQ_data_test))

In [5]:
pmid_index_mapping = ph.pmid_index_mapping()

model.top_k=10000


Load /backup/saved_models/pmid_index_mapping.p


In [49]:
query_max_index = 25

#print(bioASQ_data_test[query_index]["body"])

query = bioASQ_test_queries_tokenized[:query_max_index]

predict = model.predict(query)[:1000,:]

#flat_relevant_doc_index = [( pmid_index_mapping[y] for y in x ) for x in bioASQ_test_expectation[:query_index]]

#flat_relevant_doc_index = set(map (lambda x: pmid_index_mapping[x],bioASQ_test_expectation[:query_index]))

DEBUG: doc_awe_emb shape (18824355, 200)
DEBUG: queries_awe_emb transpose shape (200, 25)
DEBUG: doc_norm shape (18824355, 1)
DEBUG: queries_norm shape (1, 25)
DEBUG: norm shape (18824355, 25)
DEBUG: dot_product shape (18824355, 25)
predict the top 2500
Sorting time: 32.591822147369385


In [52]:
predicted_embeddings_docs = []
embeddings_docs_rel = []

for i in range(query_max_index):
    query_emb = sum(map(lambda x: model.embedding[x] ,query[i]))/len(query[i])
    query_emb = query_emb/np.linalg.norm(query_emb)
    #query 
    #queries_emb.append(np.zeros(200)+0.00001)
    
    #documents
    predicted_embeddings_docs.extend([ model.doc_awe[doc_id]/np.linalg.norm(model.doc_awe[doc_id])-query_emb for doc_id in predict[:,i]])
    
    #pos_doc
    #relevant_doc_index = list(map (lambda x: pmid_index_mapping[x],bioASQ_test_expectation[i]))
    embeddings_docs_rel.extend([ model.doc_awe[doc_id]/np.linalg.norm(model.doc_awe[doc_id])-query_emb for doc_id in predict[:,i] if pmid_index_mapping.inverse[doc_id] in set(bioASQ_test_expectation[i])])


In [11]:
from sklearn.decomposition import PCA

from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
%matplotlib inline


[0.09630828 0.06484486]


In [53]:
pca = PCA(n_components=2)

pca.fit(embeddings_docs_rel+predicted_embeddings_docs)

predicted_embeddings_2d = pca.transform(predicted_embeddings_docs)
embeddings_docs_rel_2d = pca.transform(embeddings_docs_rel)
queries_emb_2d = np.zeros(embeddings_docs_rel_2d.shape)
print(pca.explained_variance_ratio_)

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [45]:
awe_predicted_embeddings_2d = predicted_embeddings_2d
awe_queries_emb_2d = queries_emb_2d
awe_embeddings_docs_rel_2d = embeddings_docs_rel_2d
#[0.15223861 0.12097761]


In [ ]:
awe_700k_predicted_embeddings_2d = predicted_embeddings_2d
awe_700k_queries_emb_2d = queries_emb_2d
awe_700k_embeddings_docs_rel_2d = embeddings_docs_rel_2d
#[0.08892894 0.06206012]


In [ ]:


figure(num=None, figsize=(12, 6), dpi=80, facecolor='w', edgecolor='k')

plt.subplot(1, 2, 1)

pred_x, pred_y = zip(*awe_predicted_embeddings_2d)
query_x, query_y = zip(*awe_queries_emb_2d)
true_x, true_y = zip(*awe_embeddings_docs_rel_2d)

plt.scatter(pred_x,pred_y,s=1, alpha =0.7)
plt.scatter(true_x,true_y,s=5)
plt.scatter(query_x,query_y,s=10,c="red")
#plt.xlim(-0.4, 0.4)
#plt.ylim(-0.4, 0.4)

plt.subplot(1, 2, 2)

pred_x, pred_y = zip(*awe_700k_predicted_embeddings_2d)
query_x, query_y = zip(*awe_700k_queries_emb_2d)
true_x, true_y = zip(*awe_700k_embeddings_docs_rel_2d)

plt.scatter(pred_x,pred_y,s=1, alpha =0.7)
plt.scatter(true_x,true_y,s=5)
plt.scatter(query_x,query_y,s=10,c="red")
#plt.xlim(-0.4, 0.4)
#plt.ylim(-0.4, 0.4)